In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_recall_fscore_support
import random
import matplotlib.pyplot as plt
import itertools

# k-nearest neighbors

This dataset was obtained from https://archive.ics.uci.edu/ml/datasets/Heart+Disease (this is a great resource for datasets to try machine learning on). It has data on patients that are and are not diagnosed with heart disease.

The attributes are:
* age: age in years 
* sex: sex (1 = male; 0 = female) 
* cp: chest pain type 
 * -- Value 1: typical angina 
 * -- Value 2: atypical angina 
 * -- Value 3: non-anginal pain 
 * -- Value 4: asymptomatic 
* trestbps: resting blood pressure (in mm Hg on admission to the hospital) 
* chol: serum cholestoral in mg/dl 
* fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
* restecg: resting electrocardiographic results 
 * -- Value 0: normal 
 * -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
 * -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
* thalach: maximum heart rate achieved 
* exang: exercise induced angina (1 = yes; 0 = no) 
* oldpeak = ST depression induced by exercise relative to rest 
* slope: the slope of the peak exercise ST segment 
 * -- Value 1: upsloping 
 * -- Value 2: flat 
 * -- Value 3: downsloping 
* ca: number of major vessels (0-3) colored by flourosopy 
* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect 
* num: diagnosis of heart disease (angiographic disease status) 
 * -- Value 0: absence.
 * -- Value 1,2,3,4: presence of heart disease


# Explore the data

Read in the data, modify the dependent variable name and plot a histogram of the ages of patients, both healthy and those with heart disease.

In [ ]:
df = pd.read_csv('cleveland.csv')

# Collapse all values 1-4 into a single value so that "num" is boolean
df = df.rename({'num':'disease'}, axis=1)
df['disease'] = df.disease.apply(lambda x: min(x, 1))
df.drop(df[df['thal'] == '?'].index, inplace=True)
display(df.head(5))

# multiple dimensions


In [ ]:
def knnPredict(data, attributes, numOfNeighbors, target='disease'):
    nn = NearestNeighbors(n_neighbors=numOfNeighbors, metric='euclidean', algorithm='auto')
    
    # Standardize
    newAttributes = []
    for attribute in attributes:
        standard = data[attribute]-data[attribute].mean()/data[attribute].std()
        standard.name = standard.name+'_s'
        newAttributes.append(standard.name)
        data = pd.concat([data,standard],axis=1)
        
    # Build underlying structure with standardized data   
    X = data[newAttributes].values
    y = data[target].values
    fit = nn.fit(X)
    
    # Choose a random patient
    i = random.randint(0,len(X)-1)
    patientX = X[i]
    patienty = y[i]
    # display('Our patient',data.iloc[i])
    
    # Find the k nearest neighbors, not including self
    distances, indices = fit.kneighbors([patientX],numOfNeighbors+1)
    nbrs = data.iloc[indices[0]]
    # Delete self from data
    nbrs = nbrs.drop(data.iloc[i].name)
    # display(nbrs)
    
    # Count the number of neighbors that have target
    have = nbrs[nbrs[target] == 0].count()[target]
    # Count the number of neighbors that DON'T have target
    dontHave = nbrs[nbrs[target] == 1].count()[target]
    # print('Have: {}\ndontHave: {}'.format(have, dontHave))
    
    # Predict that our random patient is like the majority of its neighbors
    predict = 0 if (dontHave > have) else 1
    # According to the records, did our patient have the target
    actual = 0 if (patienty == 0) else 1
    success = predict == actual
    # print('Sucess:',success)
    return success, patienty == 0

In [ ]:
def testMeSenpai(data, attributes, numOfNeighbors, numOfTests=1, target='disease'):
    y_pred = []
    y_true = []
    for test in range(numOfTests):
        yPred, yAct = knnPredict(data, attributes, numOfNeighbors)
        y_pred.append(yPred)
        y_true.append(yAct)
    return y_pred ,y_true

In [ ]:
def crossValidate(data, attributes, numOfNeighbors, numOfTests, target='disease', level = 10):
    folds = []
    for i in range(level):
        folds.append(data.sample(frac = 1/level))
        
    # Train missing one fold
    for i in range(level-1):
        temp = pd.concat([data,folds[i]])
        temp = temp.drop_duplicates(keep=False)
        
        y_pred,y_true = testMeSenpai(temp, attributes, numOfNeighbors, numOfTests, target='disease')
        (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true,zero_division=0)
        print(p,r,f,s,'\n')
        
df1 = df.copy()
k = 5
numOfTests = 5
attributes = ['age','trestbps']

crossValidate(df1,attributes,k,numOfTests)
# y_pred, y_true = testMeSenpai(df1,attributes,k,numOfTests)
# (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true)
# display(p,r,f,s)

In [ ]:
def testAgainstTest(testData,attributes,k):
    y_pred, y_true = testMeSenpai(testData,attributes,k)
    (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true)
    return p,r,f,s

The model is the value of k (nearest neigbors) and set of attributes

## The challenge

In [ ]:
# Day of the challenge set, all we need to do is change this file name and run it
new_df = pd.read_csv('cleveland-test-sample.csv')

# Collapse all values 1-4 into a single value so that "num" is boolean
new_df = new_df.rename({'num':'disease','Unnamed: 0':'id'}, axis=1)
new_df['disease'] = new_df.disease.apply(lambda x: min(x, 1))

# Build a bigger data set to 
temp = pd.concat([df,new_df]).reset_index()


## chosen features

In [ ]:


chosen_features = ['cp', 'trestbps', 'chol', 'restecg', 'thalach', 'exang', 'thal', 'age']
numFeatures = 5
attributes = ['age','trestbps','chol', 'thalach']
# standardizing noncatagorical data
newAttributes = []
data1 = df.copy()
for attribute in attributes:
    standard = (data1[attribute]-data1[attribute].mean())/data1[attribute].std()
    chosen_features.remove(standard.name)
    chosen_features.append(standard.name+'_s')
    standard.name = standard.name+'_s'
    newAttributes.append(standard.name)
    data1 = pd.concat([data1,standard],axis=1)
    data1.drop(attribute,axis=1,inplace=True)
def makefeaturesList(chosen_features, n):
    featureslist = list(itertools.combinations(chosen_features,n)) #
    featureslist = [list(x) for x in featureslist]
    return featureslist
# data 1 is processed

## monte carlo data split with selected features

In [ ]:
# make random split for one monte carlo cross validation run
def monteCarloCVSplit(data : pd.DataFrame , level = 10):
    CVtest = data.sample(frac = 1/level)
    CVtrain = data.drop(CVtest.index)
    return CVtrain, CVtest

### run knn on given attributes and CV split

In [ ]:
def knnPredictions(CVtrain, CVTest, attributes, k, target='disease'):
    nn = NearestNeighbors(n_neighbors=k , metric='euclidean', algorithm='auto')
    
    # Build underlying structure with standardized data   
    X = CVtrain[attributes].values
    y = CVtrain[target].values
    
    testX = CVTest[attributes].values
    testy = CVTest[target].values
    fit = nn.fit(X) # fits training data to model
    
    # nbrs = neighbors
    #this fit finds the nearest neigbors from the training set to the test set 
    distances, nbrs = fit.kneighbors(testX, n_neighbors = k) 
    
    # Count the number of neighbors that have target

    diseasdedNbrCount = []
    for nbrList in nbrs:
        diseasdedNbrCount.append([1 if y[nbr] == 1 else 0 for nbr in nbrList])
    
    predictions = []
    for l in diseasdedNbrCount:
        numOfDiseased = sum(l)
        if numOfDiseased > k/2:
            predictions.append(1)
        else:
            predictions.append(0)
    # print(predictions,'\n',diseasdedNbrCount)
    return predictions , testy
    

In [ ]:
CVtrain1, CVtest1 = monteCarloCVSplit(data1)
featureslist = makefeaturesList(chosen_features, 7)
preds, actuals = knnPredictions(CVtrain1, CVtest1, featureslist[0] , 5)

(p,r,f,s) = precision_recall_fscore_support(preds,actuals,zero_division=0,average='binary')
f1 = (p*r)/(p + r)
print(p,r,f1)

make changes

In [ ]:
# traffic = pd.read_csv('traffic-Brazil.csv',sep=';')
# traffic = traffic.rename(columns={'Slowness in traffic (%)':'slowness'})
# traffic['slowness'] = traffic['slowness'].str.replace(',','.').astype(float)
# trafficMean = traffic['slowness'].mean()
# traffic['highTraffic'] = traffic['slowness'] > trafficMean

# attributes = ['Immobilized bus', 'Broken Truck','Vehicle excess','Accident victim']
# k = 6
# target = 

power = pd.read_csv('city_power_consumption.csv')
power = power.rename(columns={'Zone 1 Power Consumption':'Zone_1_Power'})
powerMean = power['Zone_1_Power'].mean()
power['highPowerUse'] = power.Zone_1_Power>powerMean
# power.plot(power['Temperature'])
# sns.lineplot(data = power, x = 'Temperature')

In [ ]:
attributes = ['Temperature','Humidity','Wind Speed','Zone 2  Power Consumption']
k = 50
target = 'highPowerUse'
# display(power)
# display(renameMe(power,attributes,k,target = target))
# display(testMeSenpai(power,attributes,k,numOfTests = 10,target = target))
# crossValidate(power,attributes,k,numOfTests,target=target)

In [ ]:
sns.scatterplot(data = power.sample(frac=.01), x = 'Temperature', y = 'Zone_1_Power')

In [ ]:
sns.scatterplot(data = power.sample(frac=.01), x = 'Humidity', y = 'Zone_1_Power')

In [ ]:
# Day of the challenge set, all we need to do is change this file name and run it
new_df = pd.read_csv('cleveland-test-sample.csv')

# Collapse all values 1-4 into a single value so that "num" is boolean
new_df = new_df.rename({'num':'disease','Unnamed: 0':'id'}, axis=1)
new_df['disease'] = new_df.disease.apply(lambda x: min(x, 1))

# Build a bigger data set to 
temp = pd.concat([df,new_df]).reset_index()